In [52]:
#Step 1: Import modules and connect to the database
import pandas as pd
import numpy as np
import pyodbc
import warnings
warnings.filterwarnings('ignore')

from IPython.display import HTML
config = dict(server='ABR-RIPLEYA-D1',
    port=      1433,
    database=  'neumodxRawDataDB',
    )

conn_str = ('SERVER={server};' +
            'Database={database};' +
            'TRUSTED_CONNECTION=yes')

conn = pyodbc.connect(
    r'DRIVER={SQL Server};' +
    conn_str.format(**config)
    )

In [53]:
#Step 2: Write or generate a query for data acquisition
##Function to Get Channel Data (i.e. Green Raw, Red Normalized etc.)
environment='4PlexV2_RPT-6224'

def getSQLData_Channel(color, environment):
    query = ("Select * from [dbo].[channel_summary]"+
             "where [Channel] = '"+color+"' AND [Environment] like '%"+environment+"%'")
    print("Query:", query)
    df = pd.read_sql(query, conn)
    return df



In [72]:
#Step 3: Perform the query on the database and return a pandas dataframe into a channel dictionary, format dataframes
channel_list = ['Yellow','Green','Red','Orange','Far_Red']

channel_dict={}
for channel in channel_list:
     newDF = getSQLData_Channel(channel,'4PlexV2_RPT-6224')
     newDF['Sample ID Prefix'] = newDF['Sample ID'].str.replace('\d+','') 
     newDF['Start Date'] = newDF['Start Time'].dt.date
     newDF['Protocol'] = newDF['File Source'].str[52:60]
     newDF=newDF.loc[:,['File Source',
                        'Protocol',
                        'Target Setting Specimen Type',
                        'Sample ID',
                        'Sample ID Prefix',
                        'Start Date',
                        'Test Guid',
                        'Replicate Result',
                        'Assay Name',
                        'Assay Version',
                        'Localized Result',
                        'Ct',
                        'End Point Fluorescence',
                        'EPR',
                        'Max Peak Height',
                        'Flags',
                        'Baseline Slope']]
     channel_dict[channel] = newDF

     index_list = ['File Source',
                    'Protocol',
                    'Target Setting Specimen Type',
                    'Sample ID',
                    'Sample ID Prefix',
                    'Start Date',
                    'Test Guid',
                    'Replicate Result',
                    'Assay Name',
                    'Assay Version']
     channel_dict[channel].set_index(index_list, inplace=True)


     new_column_names = []
     for col in channel_dict[channel].columns:
          new_column_names.append(channel+' Target '+col)

     channel_dict[channel].columns = new_column_names

Query: Select * from [dbo].[channel_summary]where [Channel] = 'Yellow' AND [Environment] like '%4PlexV2_RPT-6224%'
Query: Select * from [dbo].[channel_summary]where [Channel] = 'Green' AND [Environment] like '%4PlexV2_RPT-6224%'
Query: Select * from [dbo].[channel_summary]where [Channel] = 'Red' AND [Environment] like '%4PlexV2_RPT-6224%'
Query: Select * from [dbo].[channel_summary]where [Channel] = 'Orange' AND [Environment] like '%4PlexV2_RPT-6224%'
Query: Select * from [dbo].[channel_summary]where [Channel] = 'Far_Red' AND [Environment] like '%4PlexV2_RPT-6224%'


In [76]:
#Step 4: Merge existing dataframes 
mergeDB = pd.concat([channel_dict[channel] for channel in channel_dict],axis=1)
mergeDB.reset_index(inplace=True)

In [78]:
#Step 5: Add expected results columns
mergeDB
for channel in channel_list:
    mergeDB[channel+' Target Expected Results']= np.nan

In [80]:
#Step 6: Export final dataframe to CSV format
mergeDB.to_csv('Sample_Key_4-plex.csv')